## TO DO

2018-11-07

* Should draw the numbers at the correct positions and orientations within the TPS.
* Then burn those numbers in to the dose distribution as a verification of the orientation.

* Rescan maxphan
* Replan
* Remeaure film
* Rescan film
* Compare doses and spatial accuracy

In [ ]:
import os
from glob import glob

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from scipy.interpolate import RegularGridInterpolator

import imageio

import pydicom

In [ ]:
%load_ext autoreload
%autoreload 2

from pymedphys.dicom import (
    load_xyz_from_dicom, load_dose_from_dicom,
    get_structure_aligned_cube,
    pull_structure, get_dose_grid_structure_mask
)

from pymedphys.film import get_interpolated_dose
from pymedphys.geometry import plot_cube, cubify_cube_definition

from pymedphys.plt import pcolormesh_grid

In [ ]:
data_dir = r'S:\Physics\Programming\data\FilmProcessing\SRS\experimentation\dcat'

In [ ]:
save_path_2_4 = os.path.join(data_dir, 'dose_2_4.png')
save_path_1_3 = os.path.join(data_dir, 'dose_1_3.png')

In [ ]:
struct_filepath = glob(os.path.join(data_dir, '*StrctrSets.dcm'))[0]
struct_filepath

In [ ]:
dose_filepath = glob(os.path.join(data_dir, '*Dose.dcm'))[0]
dose_filepath

In [ ]:
dcm_dose = pydicom.dcmread(dose_filepath, force=True)
dcm_struct = pydicom.dcmread(struct_filepath, force=True)

In [ ]:
dose = load_dose_from_dicom(dcm_dose)
x_dose, y_dose, z_dose = load_xyz_from_dicom(dcm_dose)

In [ ]:
box_structure_name = 'ANT Box'

In [ ]:
contours = pull_structure(box_structure_name, dcm_struct)

# concatenated_contours = [
#     np.concatenate(contour_coord)
#     for contour_coord in contours
# ]

# bounds = [
#     (np.min(concatenated_contour), np.max(concatenated_contour))
#     for concatenated_contour in concatenated_contours
# ]

# x0 = np.array([
#     (bounds[1][0], bounds[0][0], bounds[2][1]),
#     (bounds[1][0], bounds[0][1], bounds[2][1]),
#     (bounds[1][1], bounds[0][0], bounds[2][1])
# ])

# cube_definition = cubify_cube_definition(x0)
# cube = cubify_cube_definition(cube_definition)

# ax = plot_cube(cube)

# for x, y, z in zip(box_x, box_y, box_z):
#     ax.plot(x, y, z, 'b-')

In [ ]:
cube_definition_array, vectors = get_structure_aligned_cube(box_structure_name, dcm_struct)

ax = plot_cube(cube_definition_array)

for x, y, z in zip(contours[0], contours[1], contours[2]):
    ax.plot(x, y, z, 'b-')
    

cube_definition_array

In [ ]:
# cube_definition_array

In [ ]:
ax = plot_cube(cube_definition_array)
colour = iter(['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'])

structure_names = ['Number1', 'Number2', 'Number3', 'Number4']
for structure_name in structure_names:
    this_colour = next(colour)
    struct_x, struct_y, struct_z = pull_structure(structure_name, dcm_struct)
    for x, y, z in zip(struct_x, struct_y, struct_z):
        ax.plot(x, y, z, c=this_colour)

# ax.legend()

In [ ]:
masks = {
    structure_name: get_dose_grid_structure_mask(structure_name, dcm_struct, dcm_dose)
    for structure_name in structure_names
}

In [ ]:
burn_map = {
    'Number1': 5,
    'Number2': 5,
    'Number3': 5,
    'Number4': 5
}

for structure_name, mask in masks.items():
    dose[mask] = burn_map[structure_name]

In [ ]:
mask_1_or_3 = masks['Number1'] | masks['Number3']
mask_1_or_3 = masks['Number2'] | masks['Number4']

In [ ]:
# dcm_struct.ROIContourSequence[0].ContourSequence[0].ContourData

In [ ]:
# struct_x, struct_y, struct_z = pull_structure('Number1', dcm_struct)
# struct_y

In [ ]:



# mask = get_dose_grid_structure_mask('Number1', dcm_struct, dcm_dose)

In [ ]:
vectors

In [ ]:
vertices_1_3 = (
    0.5 * vectors[0] + cube_definition_array[3],
    0.5 * vectors[0] + -vectors[2] + cube_definition_array[3],
    0.5 * vectors[0] + -vectors[2] + vectors[1] + cube_definition_array[3],
    0.5 * vectors[0] + vectors[1] + cube_definition_array[3],
)

vertices_2_4 = (
    0.5 * -vectors[2] + cube_definition_array[3],
    0.5 * -vectors[2] + vectors[1] + cube_definition_array[3],
    0.5 * -vectors[2] + vectors[1] + vectors[0] + cube_definition_array[3],
    0.5 * -vectors[2] + vectors[0] + cube_definition_array[3],
)

In [ ]:
side_length = np.sqrt(np.sum(vectors[0]**2))

film_side_length = 56.73  # measured manually on the film image
resolution = 0.1
number_of_points = int(film_side_length / resolution + 1)
margin = (side_length - film_side_length) / 2

margin_ratio = margin / side_length
margin_ratio

In [ ]:
coords_grid_1_3 = np.array([
    [
        vertices_1_3[0] + -vectors[2] * step_i + vectors[1] * step_j
        for step_i in np.linspace(1 - margin_ratio, margin_ratio, number_of_points)
    ]
    for step_j in np.linspace( 1 - margin_ratio, margin_ratio, number_of_points)
])

coords_grid_2_4 = np.array([
    [
        vertices_2_4[0] + vectors[1] * step_i + vectors[0] * step_j
        for step_i in np.linspace(margin_ratio, 1 - margin_ratio, number_of_points)
    ]
    for step_j in np.linspace(margin_ratio, 1 - margin_ratio, number_of_points)
])

In [ ]:
dose_ij_indexing = np.swapaxes(dose, 0, 1)

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.RegularGridInterpolator.html
dose_interpolation = RegularGridInterpolator(
    (x_dose, y_dose, z_dose), dose_ij_indexing)

In [ ]:
dose_1_3 = np.swapaxes(get_interpolated_dose(coords_grid_1_3, dose_interpolation), 0, 1)[::-1,::]
dose_2_4 = get_interpolated_dose(coords_grid_2_4, dose_interpolation)[::-1,::]

In [ ]:
# Burn orientation and scale markers into the image

max_val = 6.5
min_val = 0

dose_2_4[0:11,0:41] = min_val
dose_2_4[0:10,0:40] = max_val

dose_2_4[-41::,-11::] = max_val
dose_2_4[-40::,-10::] = min_val


dose_1_3[0:11,0:41] = min_val
dose_1_3[0:10,0:40] = max_val

dose_1_3[-41::,-11::] = max_val
dose_1_3[-40::,-10::] = min_val

In [ ]:
film_edge_vector = coords_grid_1_3[0,-1,:] - coords_grid_1_3[-1,-1,:]
calced_film_edge = np.sqrt(np.sum(film_edge_vector**2))
calced_film_edge

In [ ]:
dim = np.shape(coords_grid_1_3)

film_x = np.linspace(0, calced_film_edge, dim[0])
film_y = np.linspace(0, calced_film_edge, dim[1])

x, y = pcolormesh_grid(film_x, film_y, resolution)

plt.figure(figsize=(12,10))
plt.pcolormesh(x, y, dose_1_3)
plt.gca().invert_yaxis()
plt.colorbar()

plt.axis('equal')

In [ ]:
dim = np.shape(coords_grid_2_4)

film_x = np.linspace(0, calced_film_edge, dim[0])
film_y = np.linspace(0, calced_film_edge, dim[1])

x, y = pcolormesh_grid(film_x, film_y, resolution)

plt.figure(figsize=(12,10))
plt.pcolormesh(x, y, dose_2_4)
plt.gca().invert_yaxis()
plt.colorbar()

plt.axis('equal')

In [ ]:
imageio.imwrite(save_path_2_4, dose_2_4)

In [ ]:
imageio.imwrite(save_path_1_3, dose_1_3)